In [1]:
import torch
from ponapt.vocab import load_vocab
from ponapt.preproc import LMPreproc
from ponapt.postproc import LMPostproc
from ponapt.lm import LM
from ponapt.generation.sampler import SentenceSampler
from ponapt.generation.sentence import ScoredSentence
from ponapt.generation.beam import Beam
from ponapt.generation.search import BeamSearch

In [2]:
vocab = load_vocab('../../../mopumopu/220219/vocab.txt')
model = LM(len(vocab), 1024, 16, 4096, 0, 0, 0, 0, 6, padding_idx = 0, max_len = 256)
model.load_state_dict(torch.load('../../../mopumopu/220219/checkpoints/lm.200.pt', map_location = 'cpu'))
model = model.cuda()
model.eval()
preproc = LMPreproc()
postproc = LMPostproc()
search = BeamSearch(model, vocab, width = 4, max_len = 30)

In [3]:
def soweli_search(sent):
    sent = preproc(sent)
    sent = [vocab(word) for word in sent.split()]
    sent = ScoredSentence(vocab, [0] * len(sent), sent)
    beam = Beam(width = 4, sents = [sent])
    for i in range(30):
        beam = search.step(beam = beam)
        if beam.terminated():
            break
    sent = beam.store[0].sent
    sent = ' '.join([vocab[word] for word in sent][:-1])
    sent = postproc(sent)
    return sent

In [8]:
lst = [
    'ma tomo Tokijo li ma tomo lawa pi ma',
    'ma tomo Lanten li ma tomo lawa pi ma',
    'ma tomo Pelin li ma tomo lawa pi ma',
    'ma tomo Paki li ma tomo lawa pi ma',
    'ma tomo Pesin li ma tomo lawa pi ma',
    'ma tomo Solu li ma tomo lawa pi ma',
    'ma tomo Wasinton li ma tomo lawa pi ma']
for prefix in lst:
    pred = soweli_search(prefix)
    print(pred)

ma tomo Tokijo li ma tomo lawa pi ma Nijon.
ma tomo Lanten li ma tomo lawa pi ma Inli.
ma tomo Pelin li ma tomo lawa pi ma Tosi.
ma tomo Paki li ma tomo lawa pi ma Kanse.
ma tomo Pesin li ma tomo lawa pi ma Sonko.
ma tomo Solu li ma tomo lawa pi ma Wensa.
ma tomo Wasinton li ma tomo lawa pi ma Montenlija.


In [11]:
lst = [
    'ma tomo lawa pi ma Nijon li ma tomo',
    'ma tomo lawa pi ma Inli li ma tomo',
    'ma tomo lawa pi ma Tosi li ma tomo',
    'ma tomo lawa pi ma Kanse li ma tomo',
    'ma tomo lawa pi ma Sonko li ma tomo',
    'ma tomo lawa pi ma Anku li ma tomo',
    'ma tomo lawa pi ma Mewika li ma tomo']
for prefix in lst:
    pred = soweli_search(prefix)
    print(pred)

ma tomo lawa pi ma Nijon li ma tomo Kipu.
ma tomo lawa pi ma Inli li ma tomo Osaka.
ma tomo lawa pi ma Tosi li ma tomo Osaka.
ma tomo lawa pi ma Kanse li ma tomo Paki.
ma tomo lawa pi ma Sonko li ma tomo Lanten.
ma tomo lawa pi ma Anku li ma tomo Osaka.
ma tomo lawa pi ma Mewika li ma tomo Kiju.


In [13]:
lst = [
    'soweli wan li lon tomo ni. soweli tu li kama tawa tomo ni. tenpo ni la soweli',
    'soweli wan li lon ma ni. soweli tu li kama tawa ma ni. tenpo ni la soweli',]
for prefix in lst:
    pred = soweli_search(prefix)
    print(pred)

soweli wan li lon tomo ni. soweli tu li kama tawa tomo ni. tenpo ni la soweli tu tu tu li lon ni.
soweli wan li lon ma ni. soweli tu li kama tawa ma ni. tenpo ni la soweli tu li lon ni.
